In [ ]:
import arcpy
import requests
import json
import datetime
import os
import io
import zipfile
import shapefile
from shapely.geometry import Point, shape
from shapely import geometry
import numpy as np
import psycopg2
import pandas as pd
import fiona
from fiona.crs import from_epsg


In [2]:
os.chdir(r'E:\ArcGIS_2\Lab2')
wksp = os.getcwd()

# Temperature

### Pull weather information

In [9]:
# Get the 153 weather stations of the Minnesota network selecting a random day
link = r'https://mesonet.agron.iastate.edu/api/1/daily.geojson?date=2023-03-15&network=MN_RWIS'
info = json.loads(requests.get(link).text)
location = []
for i in range(len(info['features'])):
    # Store the station and its coordinates
    location.append({'station': info['features'][i]['properties']['station'], 
                     'coordinates': info['features'][i]['geometry']['coordinates']})

In [24]:
# Daily weather data from 2023
url = r'https://mesonet.agron.iastate.edu/api/1/daily.geojson?network=MN_RWIS&year=2023'
weather = json.loads(requests.get(url).text)

In [25]:
# Delete all the unneeded weather variables and keep only minimum and maximum temperature
delete = [
      "tmpf_est",
      "precip",
      "precip_est",
      "max_gust",
      "snow",
      "snowd",
      "min_rh",
      "max_rh",
      "min_dwpf",
      "max_dwpf",
      "min_feel",
      "max_feel",
      "min_rstage",
      "max_rstage",
      "temp_hour",
      "max_gust_localts",
      "max_drct",
      "avg_feel", 
      "avg_sknt", 
      "vector_avg_drct", 
      "id"
]

for i in range(len(weather['features'])):
    for key in delete:
        del weather['features'][i]['properties'][key]

### Pull Minnesota boundary from MGC

In [42]:
# MN boundary
mn_url = "https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dot/bdry_state/shp_bdry_state.zip"
boundaries = requests.post(mn_url)
zipfile.ZipFile(io.BytesIO(boundaries.content)).extractall(wksp)

# Project shp
sr = arcpy.SpatialReference(4326)
arcpy.Project_management('Boundaries_of_Minnesota.shp', 'minnesota.shp', sr)

<Result 'E:\\ArcGIS_2\\Lab2\\minnesota.shp'>

### QA

In [26]:
def ranges_intersect(range1, range2):
    """
    Returns True if the two ranges intersect, False otherwise.
    Each range is a tuple of two numbers representing the minimum and maximum values of the range.
    """
    if range1[1] < range2[0] or range2[1] < range1[0]:
        return False
    else:
        return True

Note: the optimum range is 62-99 F, but a different range is used to train the code as the temperature recorded for 2023 is still below the optimum range due to the winter.

In [27]:
# Optimum temperature
lower_temp = 10 #62
upper_temp = 50 #99
opt_temp = range(lower_temp, upper_temp+1)

In [28]:
# Read in the shapefile data for Minnesota
sf = shapefile.Reader("minnesota.shp")
shapes = sf.shapes()
state_border = shapes[0]

# Create a shapely Polygon object from the state border shape
border_polygon = shape(state_border)

# Create an empty list to add the not useful readings
wrong = []

# Add all the not useful readings to a list
for i in range(len(weather['features'])):
    
    # Readings whose temp readings are None
    if weather['features'][i]['properties']['min_tmpf'] == None or weather['features'][i]['properties']['max_tmpf'] == None:
        wrong.append(weather['features'][i])
        continue
    
    # Stations outside of Minnesota
    point = Point(weather['features'][i]['geometry']['coordinates'])
    if not border_polygon.contains(point):
        wrong.append(weather['features'][i])
        continue
        
    # Readings whose min and max temp are the same. This is an indicator of wrong data
    if weather['features'][i]['properties']['min_tmpf'] == weather['features'][i]['properties']['max_tmpf']:
        wrong.append(weather['features'][i])
        continue
        
    # Readings whose temp is outside of the optimum range
    lower_limit = math.floor(weather['features'][i]['properties']['min_tmpf'])
    upper_limit = math.ceil(weather['features'][i]['properties']['max_tmpf'])
    range_temp = range(lower_limit, upper_limit)
    
    # Readings not representative of the broader region if max and min temp are similar
    if len(range_temp) == 1:
        wrong.append(weather['features'][i]) 
        continue
    
# Delete the not useful readings 
for element in wrong:
    weather['features'].remove(element)

### Monthly average temperature

In [30]:
stations = []
# Add the dictionaries to a data frame
for j in range(len(weather['features'])):
    stations.append(weather['features'][j]['properties'])
df = pd.DataFrame.from_dict(stations)

# Remove the day part from the date leaving only year and month
for i in range(len(df['date'])):
    df['date'][i] = df['date'][i][:7]
    
# Get monthly average min and max temperature for each station
grouped = df.groupby(['station', 'date', 'name']).agg('mean')
grouped.reset_index(inplace=True)

# Return data to a dictionary
mean = grouped.to_dict('records')

[30]:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [31]:
## Add the geometry to the stations
mean_tmp = []
for i in range(len(mean)):
    for j in range(len(location)):
        if mean[i]['station'] == location[j]['station']:
            mean_tmp.append({'type': 'Feature', 'properties': mean[i], 
                             'geometry': {'type': 'Point', 'coordinates': location[j]['coordinates']}})

In [33]:
# Remove monthly average temperature if outside of the optimum range
bad_data = []
for i in range(len(mean_tmp)):
    lower_limit = math.floor(mean_tmp[i]['properties']['min_tmpf'])
    upper_limit = math.ceil(mean_tmp[i]['properties']['max_tmpf'])
    range_temp = range(lower_limit, upper_limit)
    # Temperature outside the optimum range is flagged
    if ranges_intersect(opt_temp, range_temp) == False:
        bad_data.append(mean_tmp[i])
        
# Delete the not useful readings 
for element in bad_data:
    mean_tmp.remove(element)

### Stations shapefile

In [35]:
schema =  {'geometry': 'Point', 'properties': {'station': 'str', 'date': 'str', 'name': 'str', 'max_tmpf': 'float', 'min_tmpf': 'float'}}

with fiona.open("stations.shp", 'w', crs = from_epsg(4326), driver = 'ESRI Shapefile', schema = schema) as output:
    for i in range(len(mean_tmp)):
          # geometry
          point = Point(mean_tmp[i]['geometry']['coordinates'])
          # attributes
          prop = mean_tmp[i]['properties']
          # write the row (geometry + attributes in GeoJSON format)
          output.write({'geometry': geometry.mapping(point), 'properties':prop})

# DEM

In [3]:
# Retrieve DEM from MGC
dem_output = requests.post(r'https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dnr/elev_30m_digital_elevation_model/fgdb_elev_30m_digital_elevation_model.zip')
zipfile.ZipFile(io.BytesIO(dem_output.content)).extractall(wksp)

In [4]:
path = os.path.join(wksp, 'DEM.tif')

arcpy.management.CopyRaster(
    in_raster=r'/elev_30m_digital_elevation_model.gdb/digital_elevation_model_30m',
    out_rasterdataset=path,
    config_keyword="",
    background_value=None,
    nodata_value="32767",
    onebit_to_eightbit="NONE",
    colormap_to_RGB="NONE",
    pixel_type="",
    scale_pixel_value="NONE",
    RGB_to_Colormap="NONE",
    format="TIFF",
    transform="NONE",
    process_as_multidimensional="CURRENT_SLICE",
    build_multidimensional_transpose="NO_TRANSPOSE"
)

<Result 'E:\\ArcGIS_2\\Lab2\\DEM.tif'>

In [38]:
# Set input DEM
dem = "DEM.tif"

# Check the spatial reference of the DEM
desc = arcpy.Describe(dem)
sr = desc.spatialReference
print("Spatial Reference of DEM: ", sr.name)

# Check the cell size of the DEM
cellSizeX = desc.meanCellWidth
cellSizeY = desc.meanCellHeight
print("Cell Size of DEM: {} x {}".format(cellSizeX, cellSizeY))

# Check for any nodata values in the DEM
nodata = arcpy.sa.SetNull(dem, dem, "VALUE = {}".format(desc.noDataValue))
if arcpy.management.GetRasterProperties(nodata, "MAXIMUM") == 0:
    print("No nodata values found in DEM") # This is expected due since MN isn't a rectangle
else:
    print("Nodata values found in DEM")

# Check the minimum and maximum elevation values in the DEM
highest_point = [2301-50, 2301+50]
lowest_point = [602-50, 602+50]

minimum = int(arcpy.management.GetRasterProperties(dem, "MINIMUM").getOutput(0))
maximum = int(arcpy.management.GetRasterProperties(dem, "MAXIMUM").getOutput(0))

if minimum >= lowest_point[0] and minimum <= lowest_point[1]:
    print("Minimum elevation value in DEM is correct")
else:
    print(f"Minimum elevation value in DEM is {minimum} and should be close to {lowest_point}")
    
if maximum >= highest_point[0] and maximum <= highest_point[1]:
    print("Maximum elevation value in DEM is correct")
else:
    print(f"Maximum elevation value in DEM is {maximum} and should be close to {highest_point}")

Spatial Reference of DEM:  NAD_1983_UTM_Zone_15N
Cell Size of DEM: 30.0 x 30.0
Nodata values found in DEM
Minimum elevation value in DEM is correct
Maximum elevation value in DEM is correct


In [42]:
# Project DEM to WGS 1984
sr = arcpy.SpatialReference(4326)
output = os.path.join(wksp, 'DEM_projected.tif')
arcpy.ProjectRaster_management(dem, output, sr)

<Result 'E:\\ArcGIS_2\\Lab2\\DEM_projected.tif'>

In [58]:
# Increase the cell size 
arcpy.management.Resample(
    in_raster="DEM_projected.tif",
    out_raster=os.path.join(wksp, 'DEM_resampled.tif'),
    cell_size="0.03 0.03",
    resampling_type="MAJORITY"
)

<Result 'E:\\ArcGIS_2\\Lab2\\DEM_resampled.tif'>

In [57]:
# Create elevation points from DEM
arcpy.conversion.RasterToPoint(
    in_raster="DEM_resampled.tif",
    out_point_features=os.path.join(wksp, 'elevation.shp'),
    raster_field="Value"
)

<Result 'E:\\ArcGIS_2\\Lab2\\elevation.shp'>

### Land Cover Data QA

In [ ]:
os.chdir(r'C:\Users\eriks\OneDrive\Documents\ArcGIS\Projects\Arc2_Lab2')
arcpy.env.workspace = r'C:\Users\eriks\OneDrive\Documents\ArcGIS\Projects\Arc2_Lab2'
wksp = os.getcwd()

In [ ]:
# Retrieve Land Cover Data
lcover_url = "https://resources.gisdata.mn.gov/pub/gdrs/data/pub/edu_umn/base_landcover_minnesota/tif_base_landcover_minnesota.zip"
lcover_data = requests.get(lcover_url, verify=True)

In [ ]:
zipfile.ZipFile(io.BytesIO(lcover_data.content)).extractall(wksp)
lcover = "landcover_impervious_statewide2013_v2.tif"

In [ ]:
lcover_lyr = arcpy.MakeRasterLayer_management(lcover, "Land Cover MN")

# Build pyramids
arcpy.BuildPyramids_management(lcover_lyr)

# Calculate statistics
arcpy.CalculateStatistics_management(lcover_lyr)

#### Verify Source Info

In [ ]:
# Verify the source file and data type
arcpy.Describe(lcover_lyr)

In [ ]:
# Convert the Raster layer to dataset for extracting properties
lcover_dataset = "lcover_dataset.tif"
arcpy.CopyRaster_management(lcover_lyr, lcover_dataset)
arcpy.management.GetRasterProperties(lcover_dataset)

#### Verify Data is within MN boundary

In [ ]:
# MN boundary
mn_url = "https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dot/bdry_state/shp_bdry_state.zip"
boundaries = requests.post(mn_url)
zipfile.ZipFile(io.BytesIO(boundaries.content)).extractall(wksp)

# Set spatial reference and add layer
sr = arcpy.SpatialReference(4326)
boundary = arcpy.Project_management('Boundaries_of_Minnesota.shp', 'minnesota.shp', sr)

In [ ]:
# Get shapefile extent
with arcpy.da.SearchCursor(boundary , ['SHAPE@']) as cursor:
    for row in cursor:
        shape_extent = row[0].extent

# Extract raster cells within shapefile extent
out_raster = arcpy.sa.ExtractByMask(lcover_dataset, boundary)

# Check if all pixels are within the boundary
result = arcpy.GetRasterProperties_management(out_raster, "MAXIMUM")
max_val = float(result.getOutput(0))

if max_val > 0:
    print("All pixels are within the boundary.")
else:
    print("There are pixels outside the boundary.")

#### Check for missing or incorrect values

In [ ]:
# Load the raster file
ds = gdal.Open('"lcover_dataset.tif"')
band = ds.GetRasterBand(1)

# Read the raster data as a numpy array
data = band.ReadAsArray()

In [ ]:
# Check for missing values
if np.any(np.isnan(data)):
    print('There are missing values in the raster.')

# Check for negative values
if np.any(data < 0):
    print('There are negative values in the raster.')

# Check the nodata value
nodata = band.GetNoDataValue()
if nodata is None:
    print('There is no nodata value set for the raster.')
else:
    print(f'The nodata value is {nodata}.')

In [ ]:
# Downsample raster for polygon conversion
# Set cell size of output raster
cell_size = 30

# Resample raster to lower resolution
lcover_resample = arcpy.Resample_management(lcover_lyr, "lcov_resample", cell_size, "NEAREST")

In [ ]:
# Convert rasterfile to polygon for pgAdmin
arcpy.RasterToPolygon_conversion(lcover_resample,"lcover_polygon","SIMPLIFY", "Value")

### Stinkbug Data QA

#### Data Acquisition and Processing

In [ ]:
# Retrieve Stinkbugs Data
sbug_url = "https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_mda/biota_bmsb/shp_biota_bmsb.zip"
sbug_data = requests.get(sbug_url, verify=True)

In [ ]:
zipfile.ZipFile(io.BytesIO(sbug_data.content)).extractall(wksp)
sbug = "BMSBSurveyDataTable.dbf"

In [ ]:
aprx = arcpy.mp.ArcGISProject("CURRENT")
aprxMap = aprx.listMaps("Map")[0]

In [ ]:
# Import stinkbug data table
sbugtable = arcpy.TableToTable_conversion(sbug, wksp, "sbugtable")

In [ ]:
# Create point layer of data table
x_field = "Longitude"
y_field = "Latitude"
output_fc = "sbug_points"
spatial_reference = arcpy.SpatialReference(4326)
sbug_points = arcpy.management.XYTableToPoint(sbugtable, output_fc, x_field, y_field, "", spatial_reference)

#### Verify Points are within Minnesota Boundary

In [ ]:
# Run intersect tool on points and MN boundary
intersect = arcpy.Intersect_analysis([sbug_points, boundary], "point_intersect.shp")

In [ ]:
# Select points that fall within the boundary of Minnesota and create new layer
arcpy.MakeFeatureLayer_management(sbug_points, "points_lyr")
selected_points = arcpy.SelectLayerByLocation_management("points_lyr", "INTERSECT", boundary)
MN_sbugpoints = arcpy.CopyFeatures_management("points_lyr", "intersect_sbug_points")

In [ ]:
result = arcpy.GetCount_management(MN_sbugpoints)
count = int(result.getOutput(0))

In [ ]:
# Verify resulting layer only includes points within MN boundary
total_points = int(arcpy.GetCount_management(MN_sbugpoints).getOutput(0))
if count == total_points:
    print("All points are within the boundary of Minnesota.")
else:
    print("Some points are outside the boundary of Minnesota.")

#### Verify observations are within acceptable range

In [ ]:
# Set acceptable count range
range = [0, 10000]

In [ ]:
# Create empty lists to store invalid and valid observations
invalid_observations = []
valid_observations = []

In [ ]:
# Look for valid/invalid observations and append them to their respective lists
with arcpy.da.SearchCursor(MN_sbugpoints, ["Adults", "Nymphs"]) as cursor:
    for row in cursor:
        if row[0] >= range[0] and row[0] <= range[1]:
            # Add the valid observation to the list
            valid_observations.append(row)
            
            if row[1] >= range[0] and row[1] <= range[1]:
                # Add the valid observation to the list
                valid_observations.append(row)
            else:
                # Add the invalid observation to the list
                invalid_observations.append(row)
                
        else:
            # Add the invalid observation to the list
            invalid_observations.append(row)

In [ ]:
print(len(invalid_observations))

## Save to PostgreSQL database

In [3]:
# Connect to postgresql database
# Having some database connection issues now
connection = psycopg2.connect(host = '34.47.216.64',
                              port = '5432',
                              database = 'lab2',
                              user = 'postgres',
                              password = 'student',
                             )

OperationalError: could not connect to server: Connection timed out (0x0000274C/10060)
	Is the server running on host "34.27.219.64" and accepting
	TCP/IP connections on port 5432?


### Temperature (stations)

In [37]:
data = ("stations.shp")
# fields I want from shapefile
fields = ["station", "date", "name", "max_tmpf", "min_tmpf", "Shape@WKT"]

# pscopg2 connection, replace *** and *** with your values
cursor = connection.cursor()
cursor.execute("DROP TABLE IF EXISTS stations")
cursor.execute("""
    CREATE TABLE stations (
        id SERIAL,
        station VARCHAR,
        date VARCHAR,
        name VARCHAR,
        max_tmpf DOUBLE PRECISION,
        min_tmpf DOUBLE PRECISION)
""")

cursor.execute("""
    SELECT AddGeometryColumn('stations', 'geom', 4326, 'POINT', 2)
""")

# use arcpy to get attribute data, populate PostGIS using psycopg2
with arcpy.da.SearchCursor(data, fields) as da_cursor:
    for row in da_cursor:
        wkt = row[5]
        # this was tough - everything needs to be a string and text being inserted wrapped in '' including wkt
        cursor.execute("INSERT INTO stations (station, date, name, max_tmpf, min_tmpf, geom) VALUES (%s, %s, %s, %s, %s, ST_GeomFromText(%s, 4326))", (row[0], row[1], row[2], row[3], row[4], wkt))

connection.commit()

# Close database connection
connection.close()

### Elevation

In [4]:
points = ('elevation.shp')
fields_points = ['pointid', 'grid_code', "Shape@WKT"]

cursor = connection.cursor()
cursor.execute("DROP TABLE IF EXISTS elevation")
cursor.execute("""
    CREATE TABLE elevation (
        id SERIAL,
        pointid INT,
        grid_code INT)
""")

cursor.execute("""
    SELECT AddGeometryColumn('elevation', 'geom', 4326, 'POINT', 2)
""")

# use arcpy to get attribute data, populate PostGIS using psycopg2
with arcpy.da.SearchCursor(points, fields_points) as da_cursor:
    for row in da_cursor:
        wkt = row[2]
        # this was tough - everything needs to be a string and text being inserted wrapped in '' including wkt
        cursor.execute("INSERT INTO elevation (pointid, grid_code, geom) VALUES (%s, %s, ST_GeomFromText(%s, 4326))", (row[0], row[1], wkt))

connection.commit()

# Close database connection
connection.close()

### Stinkbug Observations

In [ ]:
sbugpoints = ('intersect_sbug_points.shp')
sbug_fields = ['FID',"CheckDate","Adults","Nymphs"]

cursor = connection.cursor()
cursor.execute("DROP TABLE IF EXISTS sbugpoints")
cursor.execute("""
    CREATE TABLE sbugpoints (
        id SERIAL,
        FID INT,
        CheckDate,
        Adults INT,
        Nymphs INT)
""")

cursor.execute("""
    SELECT AddGeometryColumn('intersect_sbug_points', 'geom', 4326, 'POINT', 2)
""")

# use arcpy to get attribute data, populate PostGIS using psycopg2
with arcpy.da.SearchCursor(sbugpoints, sbug_fields) as da_cursor:
    for row in da_cursor:
        wkt = row[2]
        # this was tough - everything needs to be a string and text being inserted wrapped in '' including wkt
        cursor.execute("INSERT INTO elevation (FID,CheckDate,Adults,Nymphs, geom) VALUES (%s, %s, ST_GeomFromText(%s, 4326))", (row[0], row[1], row[2], row[3], wkt))

connection.commit()

# Close database connection
connection.close()

### Land Cover

In [ ]:
# Could not successfully convert raster data to shapefile
# Data not uploaded to database